# 3. スマートフォンの伝送データをCSVで保存してみよう

このチュートリアルでは、iOSアプリケーション  **Visual M2M Motion** （以下 VM2M Motionと呼びます）を使用してサーバーにアップロードしたデータを、 **intdash SDK for Python** （以降 intdash SDK と呼びます） を使用してCSV形式でローカルに保存します。このケースでは、以下を中心に解説します。

- スマートフォンのセンサーデータ（バイナリデータ）を数値に変換するための信号定義(※)を登録する
- 登録した信号定義を使ってデータを取得する
- 取得した時系列データを`pandas.DataFrame`に変換し、CSVファイルとして保存する

```
 信号定義とは、バイナリ形式のデータから一部を切り出し、数値や文字列など値データとして読み出す処理を定義したものです。
 この処理はサーバー側で実行されます。変換の種類に関しては、intdash.Conversionの派生クラスのドキュメントを参照してください     
(https://docs.intdash.jp/sdk/python/latest/ja/signals.html#intdash.Conversion)
```


## 3.0 事前準備
本シナリオを実施する前に、以下を用意する必要があります。

- 計測用のエッジ
- Visual M2M Motionを使用してアップロードされた計測（スマートフォンのセンサーデータを含む）
- スマートフォンのセンサーデータに紐づく信号定義


### 使用データ
本シナリオでは、以下のデータをサーバー側に準備する必要があります。  

|データ項目|本シナリオで登場するデータ名|
|:---|:---|
|時系列データを登録するエッジ|edge1|
|信号定義(※)| `sp_ACCX`, `sp_ACCY`, `sp_ACCZ`, `sp_Yaw`, `sp_Pitch`, `sp_Roll`|

(※) 登録方法については、以下の手順を確認してください。

### 計測データの作成と時系列データのアップロード
**Visual M2M Motion** を使用してデータをアップロードします。 計測を登録後、計測が新しく生成されたことを **Data Visualizer** で確認してください。

アップロード方法については、 **intdashチュートリアル2A：スマートフォンの伝送データをリアルタイムに可視化してみよう** を参照してください。

Data Visualizerの[Stored Data]に計測が表示されていれば完了です。  
この後、この計測の時系列データを取得し、CSV形式で保存します。

<img src="https://user-images.githubusercontent.com/70192465/94385045-103f7280-017f-11eb-8f12-30df59079069.png">

### パッケージのimportとクライアントの生成
`intdash.Client` に与える `url` は intdashサーバーの環境情報から、`username` と `password` はログイン用エッジで発行したアクセス情報を指定してください。  

In [20]:
import pandas as pd

import intdash
from intdash import timeutils

# Create client
client = intdash.Client(
    url = "https://example.intdash.jp",
    username = "edge1",
    password="password_here"
)

### 信号定義が登録されていることを確認する
サーバー側に対象の信号定義が登録されているかどうか確認します。  
登録されていない場合、次手順の **「(Option) 信号定義を登録する」** を参照してください。

In [7]:
signals = client.signals.list(label='sp')

In [8]:
for s in signals:
    print(s.label,  end=', ')

sp_ACCX, sp_ACCY, sp_ACCZ, sp_Pitch, sp_Roll, sp_Yaw, 

### (Option) 信号定義を登録する
```
Warning:
    既にサーバー側に対象の信号定義が登録されている場合、本手順はスキップしてください。
```

本シナリオでは「汎用センサー型」のデータを使用します。このデータはintdash独自の形式であるため、サーバー側でスマートフォンのセンサー型から数値型（今回はFloat型）に変換します。
この変換を行うためには、「信号定義」を登録する必要があります。
信号定義を登録するためには、以下のファイルを実行してください。

[0_create-signal-general-sensor.ipynb](../0_create-signal-general-sensor/0_create-signal-general-sensor.ipynb) 

今回は、「汎用センサー型」のうち、6軸センサーのデータに対してのみ変換定義を登録します。  

以上で事前準備は終了です。

## 3.1 計測に使用したエッジを取得する
VM2M Motionでログインした際に使用したエッジを取得します。

In [21]:
edges = client.edges.list(name='edge1')
edge1 = edges[0]

In [22]:
edge1.name

'edge1'

## 3.2 時系列データを取得する
時系列データの取得には `client.data_points` エンドポイントを使用します。  
`labels` に、事前に登録している信号定義のラベル名を指定して実行します。  
`start` および `end` は実際に保存した計測を含む時間に変更してください。

In [11]:
dps = client.data_points.list(
    edge_name='edge1',
    start=timeutils.str2timestamp('2020-07-17 00:00:00+09:00'), # change the actual time
    end=timeutils.str2timestamp('2020-07-18 00:00:00+09:00'), # change the actual time
    labels=['sp_ACCX', 'sp_ACCY', 'sp_ACCZ', 'sp_Yaw', 'sp_Pitch', 'sp_Roll'],
    limit=0
)

## 3.3 DataPointsのリストをDataFrameに変換する

以下のように指定することで、 `data_id` ごとのカラムを持つDataFrameに変換することができます。

In [12]:
from intdash import data

df = pd.DataFrame( [ {
     'time' : d.time,
       d.data_id : data.Float.from_payload(d.data_payload).value # convert binary to numbers.
    }
    for d in dps
]).groupby("time").last()

df

,sp_ACCX,sp_ACCY,sp_ACCZ,sp_Yaw,sp_Pitch,sp_Roll
time,,,,,,
2020-07-17T01:21:45.911747000Z,0.038937,0.491985,0.205379,93.873304,78.406296,3.800612
2020-07-17T01:21:45.931913000Z,0.139753,0.429696,-0.064305,93.758048,78.363416,3.929786
2020-07-17T01:21:45.952079000Z,0.102823,0.398769,-0.076985,93.788928,78.339776,3.883596
2020-07-17T01:21:45.972245000Z,0.046802,0.320168,-0.009198,93.819816,78.322112,3.858561
2020-07-17T01:21:45.992411000Z,0.009810,0.294638,-0.049083,93.846640,78.298808,3.835275
...,...,...,...,...,...,...
2020-07-17T01:22:37.276434000Z,0.295249,0.019525,0.202644,95.308896,20.844832,-8.388880
2020-07-17T01:22:37.296602000Z,0.090947,0.020039,-0.114039,95.321776,20.800294,-8.462379
2020-07-17T01:22:37.316769000Z,-0.232320,-0.185733,-0.398277,95.396584,20.714854,-8.827780


## 3.5 CSV形式で保存する
DataFrameの機能を使い、CSVファイルに変換します。

In [13]:
df.to_csv('./sample.csv')

以下のように保存されます。

<img src="https://user-images.githubusercontent.com/70192465/94385037-0c135500-017f-11eb-9667-781a067e0ec6.png">